In [4]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'dkRewe8bWqDvG_wGkUpZZWfiK76sgOynJi2_lGtlmIOW',
    'service_id': 'iam-ServiceId-f944cf31-4f89-489c-87b7-78cddfcaf29c',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_8c14de1d42f947c2812d9d92c44fc593_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('winequality-red.csv', 'default-donotdelete-pr-ijbvuypsfmigjt'))
df.take(5)


[Row(fixed acidity='7.4', volatile acidity='0.7', citric acid='0.0', residual sugar='1.9', chlorides='0.076', free sulfur dioxide='11.0', total sulfur dioxide='34.0', density='0.9978', pH='3.51', sulphates='0.56', alcohol='9.4', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.88', citric acid='0.0', residual sugar='2.6', chlorides='0.098', free sulfur dioxide='25.0', total sulfur dioxide='67.0', density='0.9968', pH='3.2', sulphates='0.68', alcohol='9.8', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.76', citric acid='0.04', residual sugar='2.3', chlorides='0.092', free sulfur dioxide='15.0', total sulfur dioxide='54.0', density='0.997', pH='3.26', sulphates='0.65', alcohol='9.8', quality='5'),
 Row(fixed acidity='11.2', volatile acidity='0.28', citric acid='0.56', residual sugar='1.9', chlorides='0.075', free sulfur dioxide='17.0', total sulfur dioxide='60.0', density='0.998', pH='3.16', sulphates='0.58', alcohol='9.8', quality='6'),
 Row(fixed acidity='7.4',

In [5]:
df.createOrReplaceTempView("df")
df = spark.sql("SELECT cast(`fixed acidity` as float) `fixed acidity`, cast(`volatile acidity` as float) `volatile acidity`,\
          cast(`citric acid` as float) `citric acid`, cast(`residual sugar` as float) `residual sugar`, \
          cast(`chlorides` as float) `chlorides`, cast(`free sulfur dioxide` as float) `free sulfur dioxide`, \
          cast (`total sulfur dioxide` as float) `total sulfur dioxide`, \
          cast(`sulphates` as float) `sulphates`, cast(`alcohol` as float) `alcohol`, IF(quality>=7, 1, 0) as `label` from df")
df.show(30)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+---------+-------+-----+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|sulphates|alcohol|label|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+---------+-------+-----+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0|     0.56|    9.4|    0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0|     0.68|    9.8|    0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|     0.65|    9.8|    0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|                60.0|     0.58|    9.8|    0|
|          7.4|             0.7|        0.0|           

In [6]:
splits = df.randomSplit([0.75, 0.25])
df_train = splits[0]
df_test = splits[1]

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

vectorAssembler = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", 
                                            "free sulfur dioxide", "total sulfur dioxide", "sulphates", "alcohol"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [8]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

In [9]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,gbt])

In [10]:
model = pipeline.fit(df_train)

In [11]:
prediction = model.transform(df_train)

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("f1") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction)

0.9541431179897399

In [13]:
prediction = model.transform(df_test)

In [14]:
binEval.evaluate(prediction)

0.8901167797719521

In [15]:
prediction.select('label').show(15)

+-----+
|label|
+-----+
|    0|
|    1|
|    0|
|    1|
|    0|
|    1|
|    0|
|    1|
|    1|
|    0|
|    1|
|    1|
|    0|
|    0|
|    0|
+-----+
only showing top 15 rows



In [16]:
df_test.select('label').show(15)

+-----+
|label|
+-----+
|    0|
|    1|
|    0|
|    1|
|    0|
|    1|
|    0|
|    1|
|    1|
|    0|
|    1|
|    1|
|    0|
|    0|
|    0|
+-----+
only showing top 15 rows



None
